In [1]:
%matplotlib inline

In [2]:
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
from glob import glob
import tables as tb

In [3]:
MODEL_NAME = 'v13'
ORIGINAL_SIZE = 650
sz = 256
bs = 1
STRIDE_SZ = 197
PATH = 'data/'

BASE_DIR = "data/train"
BASE_TEST_DIR = "data/test"
WORKING_DIR = "data/working"
IMAGE_DIR = "data/working/images/{}".format('v12')
V5_IMAGE_DIR = "data/working/images/{}".format('v5')

# ---------------------------------------------------------
# Parameters
MIN_POLYGON_AREA = 30  # 30

# ---------------------------------------------------------
# Input files
FMT_TRAIN_SUMMARY_PATH = str(
    Path(BASE_DIR) /
    Path("{prefix:s}_Train/") /
    Path("summaryData/{prefix:s}_Train_Building_Solutions.csv"))
FMT_TRAIN_RGB_IMAGE_PATH = str(
    Path(BASE_DIR) /
    Path("{prefix:s}_Train/") /
    Path("RGB-PanSharpen/RGB-PanSharpen_{image_id:s}.tif"))
FMT_TEST_RGB_IMAGE_PATH = str(
    Path(BASE_TEST_DIR) /
    Path("{prefix:s}_Test/") /
    Path("RGB-PanSharpen/RGB-PanSharpen_{image_id:s}.tif"))
FMT_TRAIN_MSPEC_IMAGE_PATH = str(
    Path(BASE_DIR) /
    Path("{prefix:s}_Train/") /
    Path("MUL-PanSharpen/MUL-PanSharpen_{image_id:s}.tif"))
FMT_TEST_MSPEC_IMAGE_PATH = str(
    Path(BASE_TEST_DIR) /
    Path("{prefix:s}_Test/") /
    Path("MUL-PanSharpen/MUL-PanSharpen_{image_id:s}.tif"))

# ---------------------------------------------------------
# Preprocessing result
FMT_RGB_BANDCUT_TH_PATH = IMAGE_DIR + "/rgb_bandcut.csv"
FMT_MUL_BANDCUT_TH_PATH = IMAGE_DIR + "/mul_bandcut.csv"

# ---------------------------------------------------------
# Image list, Image container and mask container
FMT_VALTRAIN_IMAGELIST_PATH = V5_IMAGE_DIR + "/{prefix:s}_valtrain_ImageId.csv"
FMT_VALTEST_IMAGELIST_PATH = V5_IMAGE_DIR + "/{prefix:s}_valtest_ImageId.csv"
FMT_VALTRAIN_IM_STORE = IMAGE_DIR + "/valtrain_{}_im.h5"
FMT_VALTEST_IM_STORE = IMAGE_DIR + "/valtest_{}_im.h5"
FMT_VALTRAIN_MASK_STORE = IMAGE_DIR + "/valtrain_{}_mask.h5"
FMT_VALTEST_MASK_STORE = IMAGE_DIR + "/valtest_{}_mask.h5"
FMT_VALTRAIN_MUL_STORE = IMAGE_DIR + "/valtrain_{}_mul.h5"
FMT_VALTEST_MUL_STORE = IMAGE_DIR + "/valtest_{}_mul.h5"

FMT_TRAIN_IMAGELIST_PATH = V5_IMAGE_DIR + "/{prefix:s}_train_ImageId.csv"
FMT_TEST_IMAGELIST_PATH = V5_IMAGE_DIR + "/{prefix:s}_test_ImageId.csv"
FMT_TRAIN_IM_STORE = IMAGE_DIR + "/train_{}_im.h5"
FMT_TEST_IM_STORE = IMAGE_DIR + "/test_{}_im.h5"
FMT_TRAIN_MASK_STORE = IMAGE_DIR + "/train_{}_mask.h5"
FMT_TRAIN_MUL_STORE = IMAGE_DIR + "/train_{}_mul.h5"
FMT_TEST_MUL_STORE = IMAGE_DIR + "/test_{}_mul.h5"
FMT_MULMEAN = IMAGE_DIR + "/{}_mulmean.h5"

# ---------------------------------------------------------
# Model files
MODEL_DIR = "data/working/models/{}".format(MODEL_NAME)
FMT_VALMODEL_PATH = MODEL_DIR + "/{}_val_weights.h5"
FMT_FULLMODEL_PATH = MODEL_DIR + "/{}_full_weights.h5"
FMT_VALMODEL_HIST = MODEL_DIR + "/{}_val_hist.csv"
FMT_VALMODEL_EVALHIST = MODEL_DIR + "/{}_val_evalhist.csv"
FMT_VALMODEL_EVALTHHIST = MODEL_DIR + "/{}_val_evalhist_th.csv"

# ---------------------------------------------------------
# Prediction & polygon result
FMT_TESTPRED_PATH = MODEL_DIR + "/{}_pred.h5"
FMT_VALTESTPRED_PATH = MODEL_DIR + "/{}_eval_pred.h5"
FMT_VALTESTPOLY_PATH = MODEL_DIR + "/{}_eval_poly.csv"
FMT_VALTESTTRUTH_PATH = MODEL_DIR + "/{}_eval_poly_truth.csv"
FMT_VALTESTPOLY_OVALL_PATH = MODEL_DIR + "/eval_poly.csv"
FMT_VALTESTTRUTH_OVALL_PATH = MODEL_DIR + "/eval_poly_truth.csv"
FMT_TESTPOLY_PATH = MODEL_DIR + "/{}_poly.csv"
FN_SOLUTION_CSV = "data/output/{}.csv".format(MODEL_NAME)

# ---------------------------------------------------------
# Model related files (others)
FMT_VALMODEL_LAST_PATH = MODEL_DIR + "/{}_val_weights_last.h5"
FMT_FULLMODEL_LAST_PATH = MODEL_DIR + "/{}_full_weights_last.h5"

## Preprocessing

In [4]:
# for train_path in ['data/train/AOI_2_Vegas_Train', 'data/train/AOI_3_Paris_Train', 'data/train/AOI_4_Shanghai_Train', 'data/train/AOI_5_Khartoum_Train']:
#     !python code/v5_im.py preproc_train {train_path}

In [5]:
# for train_path in ['data/train/AOI_2_Vegas_Train', 'data/train/AOI_3_Paris_Train', 'data/train/AOI_4_Shanghai_Train', 'data/train/AOI_5_Khartoum_Train']:
#     !python code/v12_im_deeplab.py preproc_train {train_path}

### Examine h5 content shape

v5

In [6]:
datapath = 'data/train/AOI_3_Paris_Train'
area_id = directory_name_to_area_id(datapath)
prefix = area_id_to_prefix(area_id)

NameError: name 'directory_name_to_area_id' is not defined

In [7]:
fn_im = FMT_VALTRAIN_IM_STORE.format(prefix)
image_list = pd.read_csv(FMT_VALTRAIN_IMAGELIST_PATH.format(prefix=prefix)).ImageId.tolist()
with tb.open_file(fn_im, 'r') as f:
    image_id = image_list[1]
    im = np.array(f.get_node('/' + image_id))
    print(im.shape)
    plt.imshow(im)

NameError: name 'prefix' is not defined

Conclusion: 255x255 scaled

v12

In [ ]:
fn_im = FMT_VALTRAIN_MUL_STORE.format(prefix)
image_list = pd.read_csv(FMT_VALTRAIN_IMAGELIST_PATH.format(prefix=prefix)).ImageId.tolist()
with tb.open_file(fn_im, 'r') as f:
    image_id = image_list[2]
    im = np.array(f.get_node('/' + image_id))
    print(im.shape)
    i = 5
    plt.imshow(im[..., i:i+3])

### Overload

In [ ]:
import sys
sys.path.insert(0, 'code')

In [ ]:
from v9s_deeplab import *

In [ ]:
# aug_tfms = [RandomRotateZoom(9, 0.18, 0.1, tfm_y=TfmType.CLASS), RandomLighting(0.05, 0.1, tfm_y=TfmType.CLASS), RandomDihedral(tfm_y=TfmType.CLASS)]

aug_tfms=transforms_top_down

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
# memory dataset
def get_dataset(datapath):
    area_id = directory_name_to_area_id(datapath)
    prefix = area_id_to_prefix(area_id)
    trn_x, trn_y = get_valtrain_data(area_id)
    val_x, val_y = get_valtest_data(area_id)
    return (trn_x,trn_y), (val_x,val_y)


In [ ]:
(trn_x,trn_y), (val_x,val_y) = get_dataset(datapath)

In [ ]:
class ArraysSingleDataset(ArraysDataset):
    def get_c(self): return 1

In [ ]:
sys.path.insert(0, 'deeplab/pytorch-deeplab-resnet')
import deeplab_resnet

In [ ]:
class Deeplab():
    def __init__(self,model,name='deeplab'):
        self.model,self.name = model,name

    def get_layer_groups(self, precompute):
        c = list(children(self.model))
        return [c]

In [ ]:
def get_deeplab(NoLabels=1):
    model = deeplab_resnet.Res_Deeplab(NoLabels)

    saved_state_dict = torch.load('deeplab/pytorch-deeplab-resnet/data/MS_DeepLab_resnet_trained_VOC.pth')
    if NoLabels != 21:
        for i in saved_state_dict:
            #Scale.layer5.conv2d_list.3.weight
            i_parts = i.split('.')
            if i_parts[1]=='layer5':
                saved_state_dict[i] = model.state_dict()[i]

    model.load_state_dict(saved_state_dict)
    model.float()
    model.eval() # use_global_stats = True
    return Deeplab(model)
#     return model

In [ ]:
def get_md_model(datapath):
    area_id = directory_name_to_area_id(datapath)
    prefix = area_id_to_prefix(area_id)
    tfms = tfms_from_model(resnet101, sz, crop_type=CropType.NO, tfm_y=TfmType.CLASS, aug_tfms=aug_tfms)
    datasets = ImageData.get_ds(ArraysSingleDataset, (trn_x,trn_y), (val_x,val_y), tfms)
    md = ImageData('data', datasets, bs, num_workers=8, classes=None)
#     denorm = md.trn_ds.denorm

    if not Path(MODEL_DIR).exists():
        Path(MODEL_DIR).mkdir(parents=True)

    logger.info("Instantiate deeplab model")
    model = get_deeplab()
    return md, model

In [ ]:
# def jaccard_coef(y_true, y_pred):
#     smooth = 1e-12
#     intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
#     jac = (intersection + smooth) / (sum_ - intersection + smooth)
#     return K.mean(jac)


# def jaccard_coef_int(y_true, y_pred):
#     smooth = 1e-12
#     y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#     intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
#     sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
#     jac = (intersection + smooth) / (sum_ - intersection + smooth)
#     return K.mean(jac)

In [ ]:
from torch import Tensor
def jaccard_coef(y_true, y_pred):
    smooth = 1e-12
    intersection = Tensor.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = Tensor.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return Tensor.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    smooth = 1e-12
    y_pred_pos = Tensor.round(K.clip(y_pred, 0, 1))
    intersection = Tensor.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = Tensor.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return Tensor.mean(jac)

In [ ]:
md, model = get_md_model(datapath)

In [ ]:
learn = ConvLearner(md, model)
learn.opt_fn=optim.Adam
learn.crit=F.binary_cross_entropy
learn.metrics=[jaccard_coef, jaccard_coef_int]

In [ ]:
t = md.trn_ds.get_x(0); t.shape

In [ ]:
model.model(Variable(torch.FloatTensor(t[None])))

In [ ]:
t = next(iter(md.trn_dl))

In [ ]:
t

In [ ]:
learn.lr_find()
learn.sched.plot()